In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import json
from pandas import json_normalize
import pprint
import requests
import re

### API datatourisme

In [2]:
result = requests.get("https://diffuseur.datatourisme.fr/webservice/74322d08d00eb7ed7319d5b7abb0a0bf/d5029c70-f4a5-4776-876b-191d047735af")

In [3]:
print(result)

<Response [200]>


In [118]:
data = result.json()

In [119]:
data.keys()

dict_keys(['@context', '@graph'])

### Création dataframe avec @graph

In [120]:
pd.set_option('display.max_columns', None)

In [121]:
data_graph = pd.DataFrame(data["@graph"])
data_graph.head(5)


,@id,dc:identifier,@type,rdfs:comment,rdfs:label,owl:topObjectProperty,availableLanguage,hasArchitecturalStyle,hasAudience,hasBeenCreatedBy,hasBeenPublishedBy,hasClientTarget,hasContact,hasDescription,hasMainRepresentation,hasRepresentation,hasTheme,hasTranslatedProperty,isLocatedAt,lastUpdate,lastUpdateDatatourisme,reducedMobilityAccess,meta:fingerprint,meta:hasFluxIdentifier,meta:hasOrganizationIdentifier,meta:sourceChecksum,schema:offers,offers,hasFeature,hasReview,isEquippedWith,owl:sameAs,COVID19SpecialMeasures,isOwnedBy,allowedPersons,creationDate,hasNeighborhood,hasBookingContact,providesFoodProduct,saleOnSite,hasAdministrativeContact,schema:endDate,schema:startDate,takesPlaceAt,dc:date,guided,hasManagementContact,hasGeographicReach,hasCommunicationContact,providesCuisineOfType
0,https://data.datatourisme.fr/10/000283ba-f94b-3bce-8f57-e5c10bec6cd4,947001245,"[ArcheologicalSite, CulturalSite, PlaceOfInterest, PointOfInterest, urn:resource]","{'@value': 'Les ruines de la Villa du Grosswald sont en bordure d'un plateau calcaire et dominent la vallée du Fusslach. La villa romaine s'insérait dans un environnement complexe, à proximité d'une voie, d'une source, d'autres villas et d'un vicus.', '@language': 'fr'}","{'@value': 'VILLA GALLO-ROMAINE DU GROSSWALD', '@language': 'fr'}","{'@id': 'data:be3a3647-4362-38d2-a2c5-721903c45872', '@type': 'Description', 'owl:topDataProperty': {'@value': 'Les ruines de la Villa du Grosswald sont en bordure d'un plateau calcaire et dominent la vallée du Fusslach. La villa romaine s'insérait dans un environnement complexe, à proximité d'une voie, d'une source, d'autres villas et d'un vicus.', '@language': 'fr'}, 'hasTranslatedProperty': [{'@id': 'data:1edc5b63-6043-30f0-97e4-04f3c09b985a', 'dc:contributor': 'https://api.deepl.com/v2/translate', 'dc:date': {'@value': '2022-11-15', '@type': 'xsd:date'}, 'rdf:language': 'it', 'rdf:predicate': {'@id': 'shortDescription'}, 'meta:checksum': 'e6c991e77438b0698af37a54a126819f-77bacd094135c3db9572f464165020da'}, {'@id': 'data:aea2ebbb-fcb7-3014-bfb8-beb0553406c6', 'dc:contributor': 'https://api.deepl.com/v2/translate', 'dc:date': {'@value': '2022-05-25', '@type': 'xsd:date'}, 'rdf:language': 'en', 'rdf:predicate': {'@id': 'shortDescription'}, 'meta:checksum': 'e6c991e77438b0698af37a54a126819f-0b038834dd1c6737010d6971bc20e634'}, {'@id': 'data:c8af4dab-9321-329b-bb6d-37ea3a3f5edd', 'dc:contributor': 'https://api.deepl.com/v2/translate', 'dc:date': {'@value': '2022-11-03', '@type': 'xsd:date'}, 'rdf:language': 'es', 'rdf:predicate': {'@id': 'shortDescription'}, 'meta:checksum': 'e6c991e77438b0698af37a54a126819f-ec2ee6cc7e02374c114c8e8fbafbb329'}, {'@id': 'data:db3f0678-aa1b-3465-ba27-0299e692e8f7', 'dc:contributor': 'https://api.deepl.com/v2/translate', 'dc:date': {'@value': '2022-09-21', '@type': 'xsd:date'}, 'rdf:language': 'de', 'rdf:predicate': {'@id': 'shortDescription'}, 'meta:checksum': 'e6c991e77438b0698af37a54a126819f-9d09c0b6ee139de86f3c30487aedc2eb'}, {'@id': 'data:f17ab835-89fe-3840-9bcb-1b2f1f6a89d4', 'dc:contributor': 'https://api.deepl.com/v2/translate', 'dc:date': {'@value': '2022-10-18', '@type': 'xsd:date'}, 'rdf:language': 'nl', 'rdf:predicate': {'@id': 'shortDescription'}, 'meta:checksum': 'e6c991e77438b0698af37a54a126819f-623dc7294325ae9843eb43e6f1163d3e'}], 'shortDescription': {'@value': 'Les ruines de la Villa du Grosswald sont en bordure d'un plateau calcaire et dominent la vallée du Fusslach. La villa romaine s'insérait dans un environnement complexe, à proximité d'une voie, d'une source, d'autres villas et d'un vicus.', '@language': 'fr'}}","[DE, EN]","{'@id': 'kb:Antique', '@type': ['owl:NamedIndividual', 'ArchitecturalStyle'], 'rdfs:label': {'@value': 'Antique', '@language': 'fr'}}","[{'@id': 'kb:ForAll', '@type': ['owl:NamedIndividual', 'PeopleAudience'], 'rdfs:label': {'@value': 'Tout public', '@language': 'fr'}}, {'@id': 'kb:Groups', '@type': ['owl:NamedIndividual', 'PeopleAudience'], 'rdfs:label': {'@value': 'Groupes', '@l

In [122]:
pd.set_option('display.max_rows', None)

In [123]:
nb_ligne = len(data_graph)
print('le nombre de lignes dans mon dataframe est de :', nb_ligne)

le nombre de lignes dans mon dataframe est de : 82454


### Nous allons créer une nouvelle colonne pour le lieu de l'évènement

In [124]:
data_graph['lieu'] = data_graph["rdfs:label"].str["@value"]

In [125]:
data_graph[['lieu']].head(2)

,lieu
0,VILLA GALLO-ROMAINE DU GROSSWALD
1,PARC DU TONNEAU


### Les nuls sur la colonne lieu : je décide de supprimer les lignes en NaN

In [126]:
nb_null_lieu = data_graph['lieu'].isnull().sum()

In [127]:
print(f"J'ai supprimé :", nb_null_lieu/nb_ligne*100, "% de lignes nulles")

J'ai supprimé : 0.07761903606859583 % de lignes nulles


In [128]:
data_graph = data_graph.dropna(subset=['lieu'])

### Colonne type d'évènement

In [129]:
data_graph['type'] = data_graph["@type"]

Suppression des caractères innutiles 

pourquoi doit-on lancer 2 fois cette ligne pour qu'elle fonctionne ? 

In [131]:
data_graph['type'] = data_graph['type'].replace('\[','',regex=True).astype(str)
data_graph['type'] = data_graph['type'].replace("\'"," ",regex=True).astype(str)
data_graph['type'] = data_graph['type'].replace("schema:","",regex=True).astype(str)

data_graph[['type']].head()


,type
0,"ArcheologicalSite , CulturalSite , PlaceOfInterest , PointOfInterest , urn:resource ]"
1,"Park , CulturalSite , ParkAndGarden , PlaceOfInterest , PointOfInterest , urn:resource ]"
2,"CulturalSite , PlaceOfInterest , PointOfInterest , ReligiousSite , urn:resource ]"
3,"LocalBusiness , PlaceOfInterest , PointOfInterest , TastingProvider , urn:resource ]"
4,"TouristInformationCenter , LocalTouristOffice , PlaceOfInterest , PointOfInterest , TouristInformationCenter , urn:resource ]"


In [132]:
print(f"il y a : ",data_graph['type'].isnull().sum(),"éléments null")

il y a :  0 éléments null


Sachant qu'il y a la colonne description, je considère la colonne type comme un supplément d'information dont les valeurs nuls ne gênera pas la compréhension.

### Colonne description de l'évènement

In [133]:
data_graph["description"] = data_graph["owl:topObjectProperty"].str["owl:topDataProperty"].str["@value"]
data_graph["description"] = data_graph["description"].replace('\r','',regex=True).astype(str)
data_graph["description"] = data_graph["description"].replace('\n','',regex=True).astype(str)
data_graph["description"] = data_graph["description"].replace('œ','oe',regex=True).astype(str)
data_graph["description"] = data_graph["description"].replace('â','a',regex=True).astype(str)
data_graph["description"] = data_graph["description"].replace('\"',' ',regex=True).astype(str)

In [134]:
data_graph["description"] = data_graph["description"].fillna(0)

In [135]:
data_graph[["description"]].head(10)

,description
0,"Les ruines de la Villa du Grosswald sont en bordure d'un plateau calcaire et dominent la vallée du Fusslach. La villa romaine s'insérait dans un environnement complexe, à proximité d'une voie, d'une source, d'autres villas et d'un vicus."
1,"En face du lavoir, le parc du Tonneau, situé dans la rue qui lui a donné son nom, était autrefois le jardin d’agrément de la propriété Georgel, belle demeure de notables édifiée à la fin du XVIIIe siècle.La batisse a subi des modifications au cours des décennies, mais le parc a gardé ses arbres d’ornement, ses fruitiers, son bassin et ses rocailles, qui font le charme de cet espace boisé classé. Racheté par la mairie, il a été ouvert au public en 2014 et fait partie du réseau Refuges LPO."
2,"Destinée à recevoir les Chevaliers de l'Ordre Teutonique (suite à un acte de donation fait en février 1236 par Matthieu II duc de Lorraine), l'église de la Nativité a été construite par des hospitaliers militaires qui s'étaient vu concéder une place pour batir une maison et une chapelle dans la ville. L'église de la Nativité étonne par le contraste entre son extérieur massif d'aspect défensif et l'élégance de son décor intérieur de type gothique. L'aspect d'église forte est soulignée par ses puissants contreforts qui font office d'arc boutant et par sa tour clocher hors oeuvre."
3,"Passionnés d'élevage de la race limousine et prônant une agriculture raisonnée, Kévin et Bernard mènent leur projet avec des engagements communs : assurer une qualité exemplaire et une vente en circuit court. Ce magasin est spécialisé dans la vente de produits locaux. Chaque produit du magasin est issu d'une production 100% locale implantée dans un rayon de 40 à 50 km autour de la commune de Hilbesheim. Des partenaires de confiance qui sont également animés par une exigence de qualité et des méthodes de production traditionnelles et naturelles."
4,Céline et Mélanie ont grand plaisir à vous accueillir et vous conseiller.
5,"Église de style roman, construite à la fin du XIème siècle ou au début du siècle suivant."
6,Bibliothèque municipale du réseau de lecture du Pays de Phalsbourg.Catalogue en ligne !
7,"Avec plus de 12 000 espèces en culture, le Jardin Botanique Jean-Marie Pelt est un véritable musée des plantes qui témoigne de l’extraordinaire richesse du monde végétal.Initié à partir de 1975 au sud-ouest de l’agglomération nancéienne, il est devenu un des plus importants équipements botaniques français.Au coeur du parc vallonné de 35 hectares, les différentes collections proposent de nombreux thèmes de découverte et d’émerveillement, des plantes communes de nos jardins aux espèces originaires des contrées les plus lointaines (alpinum, arboretum, roseraie, plantes médicinales, collections historique et systématique ...).Dans les serres tropicales de 2 500 m², la végétation exubérante offre un dépaysement permanent. Une simple visite se transforme vite en un véritable parcours d’explorateur.Une nouvelle application de visite permet de découvrir le Jardin Botanique de façon ludique et innovante en associant jeux, vidéos, et réalité augmentée autour de 4 parcours (adultes et enfants). Elle est téléchargeable gratuitement sur votre smartphone ou disponible sur des tablettes en location à l’accueil du jardin.Plusieurs collections spécialisées (Aracées, Orchidacées, plantes succulentes, insectivores et myrmécophiles, Broméliacées...) sont reconnues pour leur importance et leur richesse par la communauté scientifique internationale.Jardin accessible gratuitement - accès aux serres tropicales payant"
8,Petite église du 18ème siècle située sur un promontoire. Caractéristique : son clocher à bulbe du XVIIIème. Elle est entourée d'un cimetière civil et militaire.
9,"Le cimetière militaire abrite la sépulture du fils du Maréchal Foch et la sépulture du fils de René Viviani, président du Conseil (1914–1918). Dans ce cimetière, reposent également 48 français (tombes) et 1215 en ossuaire.Le monument aux morts où est inscr

In [136]:
data_graph["description"].isnull().sum()

0

### Colonne latitude

In [137]:
data_graph["latitude"] = data_graph['isLocatedAt'].str["schema:geo"].str["schema:latitude"].str["@value"]

In [138]:
data_graph[["latitude"]].head(2)

,latitude
0,49.0816164488503
1,48.6568275


Colonnes avec des Nan, si pas de coordonnées géographique, impossible de faire une géolocalisation donc suppression de ces colonnes

In [139]:
data_graph["latitude"].isnull().values.any()

True

In [140]:
print(f"il y a", data_graph["latitude"].isnull().sum(), "lignes supprimées")

il y a 5243 lignes supprimées


In [141]:
data_graph = data_graph.dropna(subset=['latitude'])

In [142]:
data_graph["latitude"]

0          49.0816164488503
1                48.6568275
2                49.4412512
3                  48.76884
4                48.1480052
5           49.135955281455
6                48.7335143
7               48.66289762
8                 48.333393
9          49.5442274645657
10         47.8776707909495
11         49.1338888888889
12               49.1256226
13               49.3332609
14         48.3006335539931
15               49.1162593
16               48.6952621
17         48.0706099961671
18                48.497942
19                 48.67269
20         48.9941028562884
21                49.504531
22               49.1054752
23         48.4886111111111
24               48.8917123
25                48.702257
26         48.0080130905611
27               49.1424747
28         48.0652894268197
29               48.9132895
30               47.9548683
31                49.203753
32               48.6849893
33         48.7694927757585
34         49.4529321919576
35                48

### Colonne longitude

In [143]:
data_graph["longitude"] = data_graph['isLocatedAt'].str["schema:geo"].str["schema:longitude"].str["@value"]

In [144]:
data_graph[["longitude"]].head(2)

,longitude
0,7.09963628523632
1,6.16705189999993


In [145]:
data_graph["longitude"].isnull().values.any()

False

### Colonne latlong

In [146]:
data_graph['latlong'] = data_graph['isLocatedAt'].str["schema:geo"].str['latlon'].str['@value']

In [147]:
data_graph[['latlong']].head(2)

,latlong
0,49.08161#7.09963
1,48.65682#6.16705


In [148]:
data_graph["latlong"].isnull().sum()

0

### Colonne ville

In [149]:
data_graph["ville"] = data_graph['isLocatedAt'].str["schema:address"].str["schema:addressLocality"]

In [150]:
data_graph["ville"] = data_graph["ville"].str.upper()

In [151]:
data_graph["ville"]= data_graph["ville"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [152]:
data_graph["ville"] = data_graph["ville"].replace('\-',' ',regex=True).astype(str)

In [153]:
data_graph[["ville"]].head(2)

,ville
0,SARREINSMING
1,VANDUVRE LES NANCY


In [154]:
data_graph["ville"].isnull().values.any()

False

In [155]:
data_graph['ville'].isnull().sum()

0

In [156]:
data_graph = data_graph.dropna(subset=['ville'])

### Colonne adresse

In [157]:
data_graph["adresse"] = data_graph['isLocatedAt'].str["schema:address"].str["schema:streetAddress"]

Pourquoi dois-je le lancer 2 fois pour que les modifs prennent effet ? 

In [158]:
data_graph["adresse"] = data_graph["adresse"].replace('\[','',regex=True).astype(str)
data_graph["adresse"] = data_graph["adresse"].replace('\]','',regex=True).astype(str)
data_graph["adresse"] = data_graph["adresse"].replace("\'",'',regex=True).astype(str)
data_graph["adresse"] = data_graph["adresse"].replace('\"','',regex=True).astype(str)

In [159]:
data_graph[["adresse"]].head(10)

,adresse
0,None
1,Rue du tonneau
2,Rue de la Tour de lHorloge
3,24 rue de Phalsbourg
4,44 Ter Habaurupt
5,Place de lEglise
6,8 rue Ackermann
7,100 Rue du Jardin Botanique
8,None
9,Rue du 131ème Régiment dInfanterie


In [160]:
data_graph[["adresse"]].isnull().sum()

adresse    0
dtype: int64

### Departement

In [161]:
data_graph.head(3)

,@id,dc:identifier,@type,rdfs:comment,rdfs:label,owl:topObjectProperty,availableLanguage,hasArchitecturalStyle,hasAudience,hasBeenCreatedBy,hasBeenPublishedBy,hasClientTarget,hasContact,hasDescription,hasMainRepresentation,hasRepresentation,hasTheme,hasTranslatedProperty,isLocatedAt,lastUpdate,lastUpdateDatatourisme,reducedMobilityAccess,meta:fingerprint,meta:hasFluxIdentifier,meta:hasOrganizationIdentifier,meta:sourceChecksum,schema:offers,offers,hasFeature,hasReview,isEquippedWith,owl:sameAs,COVID19SpecialMeasures,isOwnedBy,allowedPersons,creationDate,hasNeighborhood,hasBookingContact,providesFoodProduct,saleOnSite,hasAdministrativeContact,schema:endDate,schema:startDate,takesPlaceAt,dc:date,guided,hasManagementContact,hasGeographicReach,hasCommunicationContact,providesCuisineOfType,lieu,type,description,latitude,longitude,latlong,ville,adresse
0,https://data.datatourisme.fr/10/000283ba-f94b-3bce-8f57-e5c10bec6cd4,947001245,"[ArcheologicalSite, CulturalSite, PlaceOfInterest, PointOfInterest, urn:resource]","{'@value': 'Les ruines de la Villa du Grosswald sont en bordure d'un plateau calcaire et dominent la vallée du Fusslach. La villa romaine s'insérait dans un environnement complexe, à proximité d'une voie, d'une source, d'autres villas et d'un vicus.', '@language': 'fr'}","{'@value': 'VILLA GALLO-ROMAINE DU GROSSWALD', '@language': 'fr'}","{'@id': 'data:be3a3647-4362-38d2-a2c5-721903c45872', '@type': 'Description', 'owl:topDataProperty': {'@value': 'Les ruines de la Villa du Grosswald sont en bordure d'un plateau calcaire et dominent la vallée du Fusslach. La villa romaine s'insérait dans un environnement complexe, à proximité d'une voie, d'une source, d'autres villas et d'un vicus.', '@language': 'fr'}, 'hasTranslatedProperty': [{'@id': 'data:1edc5b63-6043-30f0-97e4-04f3c09b985a', 'dc:contributor': 'https://api.deepl.com/v2/translate', 'dc:date': {'@value': '2022-11-15', '@type': 'xsd:date'}, 'rdf:language': 'it', 'rdf:predicate': {'@id': 'shortDescription'}, 'meta:checksum': 'e6c991e77438b0698af37a54a126819f-77bacd094135c3db9572f464165020da'}, {'@id': 'data:aea2ebbb-fcb7-3014-bfb8-beb0553406c6', 'dc:contributor': 'https://api.deepl.com/v2/translate', 'dc:date': {'@value': '2022-05-25', '@type': 'xsd:date'}, 'rdf:language': 'en', 'rdf:predicate': {'@id': 'shortDescription'}, 'meta:checksum': 'e6c991e77438b0698af37a54a126819f-0b038834dd1c6737010d6971bc20e634'}, {'@id': 'data:c8af4dab-9321-329b-bb6d-37ea3a3f5edd', 'dc:contributor': 'https://api.deepl.com/v2/translate', 'dc:date': {'@value': '2022-11-03', '@type': 'xsd:date'}, 'rdf:language': 'es', 'rdf:predicate': {'@id': 'shortDescription'}, 'meta:checksum': 'e6c991e77438b0698af37a54a126819f-ec2ee6cc7e02374c114c8e8fbafbb329'}, {'@id': 'data:db3f0678-aa1b-3465-ba27-0299e692e8f7', 'dc:contributor': 'https://api.deepl.com/v2/translate', 'dc:date': {'@value': '2022-09-21', '@type': 'xsd:date'}, 'rdf:language': 'de', 'rdf:predicate': {'@id': 'shortDescription'}, 'meta:checksum': 'e6c991e77438b0698af37a54a126819f-9d09c0b6ee139de86f3c30487aedc2eb'}, {'@id': 'data:f17ab835-89fe-3840-9bcb-1b2f1f6a89d4', 'dc:contributor': 'https://api.deepl.com/v2/translate', 'dc:date': {'@value': '2022-10-18', '@type': 'xsd:date'}, 'rdf:language': 'nl', 'rdf:predicate': {'@id': 'shortDescription'}, 'meta:checksum': 'e6c991e77438b0698af37a54a126819f-623dc7294325ae9843eb43e6f1163d3e'}], 'shortDescription': {'@value': 'Les ruines de la Villa du Grosswald sont en bordure d'un plateau calcaire et dominent la vallée du Fusslach. La villa romaine s'insérait dans un environnement complexe, à proximité d'une voie, d'une source, d'autres villas et d'un vicus.', '@language': 'fr'}}","[DE, EN]","{'@id': 'kb:Antique', '@type': ['owl:NamedIndividual', 'ArchitecturalStyle'], 'rdfs:label': {'@value': 'Antique', '@language': 'fr'}}","[{'@id': 'kb:ForAll', '@type': ['owl:NamedIndividual', 'PeopleAudience'], 'rdfs:label': {'@value': 'Tout public', '@language': 'fr'}}, {'@id': 'kb:Groups', '@type': ['owl:NamedIndividu

### Region

In [162]:
data_graph['region'] = data_graph.isLocatedAt.str['schema:address'].str['hasAddressCity'].str['isPartOfDepartment'].str['isPartOfRegion'].str['rdfs:label'].str['@value'].head(1)

In [163]:
data_graph['region'].isnull().sum()

77146

### Colonne Insee

In [164]:
data_graph["code_insee"] = data_graph["isLocatedAt"].str["schema:address"].str['hasAddressCity'].str['insee']

In [165]:
data_graph[["code_insee"]].head(2)

,code_insee
0,57633
1,54547


In [166]:
len(data_graph)

77147

In [167]:
print(f"il y a :",(data_graph['code_insee'].isnull().sum())/len(data_graph)*100,"% de données vides")

il y a : 79.47943536365639 % de données vides


In [168]:
data_graph["code_insee"].isnull().sum()

61316

### Colonne date de fin

In [169]:
data_graph["date_de_fin"] = data_graph['schema:endDate'].str['@value']

In [170]:
data_graph["date_de_fin"].isnull().values.any()

True

### Changement des NaN par 2099-01-01 

In [171]:
data_graph["date_de_fin"] = data_graph["date_de_fin"].fillna("2099-01-01")

In [172]:
data_graph[["date_de_fin"]].head(2)

,date_de_fin
0,2099-01-01
1,2099-01-01


### Suppression des colonnes unitiles 

In [173]:
data_graph = data_graph[['lieu','type','description', 'latitude', 'longitude', 'latlong', 'ville', 'adresse', 'code_insee','date_de_fin']]

In [174]:
data_graph.head(10)

,lieu,type,description,latitude,longitude,latlong,ville,adresse,code_insee,date_de_fin
0,VILLA GALLO-ROMAINE DU GROSSWALD,"ArcheologicalSite , CulturalSite , PlaceOfInterest , PointOfInterest , urn:resource ]","Les ruines de la Villa du Grosswald sont en bordure d'un plateau calcaire et dominent la vallée du Fusslach. La villa romaine s'insérait dans un environnement complexe, à proximité d'une voie, d'une source, d'autres villas et d'un vicus.",49.0816164488503,7.09963628523632,49.08161#7.09963,SARREINSMING,None,57633,2099-01-01
1,PARC DU TONNEAU,"Park , CulturalSite , ParkAndGarden , PlaceOfInterest , PointOfInterest , urn:resource ]","En face du lavoir, le parc du Tonneau, situé dans la rue qui lui a donné son nom, était autrefois le jardin d’agrément de la propriété Georgel, belle demeure de notables édifiée à la fin du XVIIIe siècle.La batisse a subi des modifications au cours des décennies, mais le parc a gardé ses arbres d’ornement, ses fruitiers, son bassin et ses rocailles, qui font le charme de cet espace boisé classé. Racheté par la mairie, il a été ouvert au public en 2014 et fait partie du réseau Refuges LPO.",48.6568275,6.16705189999993,48.65682#6.16705,VANDUVRE LES NANCY,Rue du tonneau,54547,2099-01-01
2,ÉGLISE PAROISSIALE DE LA NATIVITÉ,"CulturalSite , PlaceOfInterest , PointOfInterest , ReligiousSite , urn:resource ]","Destinée à recevoir les Chevaliers de l'Ordre Teutonique (suite à un acte de donation fait en février 1236 par Matthieu II duc de Lorraine), l'église de la Nativité a été construite par des hospitaliers militaires qui s'étaient vu concéder une place pour batir une maison et une chapelle dans la ville. L'église de la Nativité étonne par le contraste entre son extérieur massif d'aspect défensif et l'élégance de son décor intérieur de type gothique. L'aspect d'église forte est soulignée par ses puissants contreforts qui font office d'arc boutant et par sa tour clocher hors oeuvre.",49.4412512,6.3562249,49.44125#6.35622,SIERCK LES BAINS,Rue de la Tour de lHorloge,57650,2099-01-01
3,CHEZ LES P'TITS PAYSANS,"LocalBusiness , PlaceOfInterest , PointOfInterest , TastingProvider , urn:resource ]","Passionnés d'élevage de la race limousine et prônant une agriculture raisonnée, Kévin et Bernard mènent leur projet avec des engagements communs : assurer une qualité exemplaire et une vente en circuit court. Ce magasin est spécialisé dans la vente de produits locaux. Chaque produit du magasin est issu d'une production 100% locale implantée dans un rayon de 40 à 50 km autour de la commune de Hilbesheim. Des partenaires de confiance qui sont également animés par une exigence de qualité et des méthodes de production traditionnelles et naturelles.",48.76884,7.23627,48.76884#7.23627,MITTELBRONN,24 rue de Phalsbourg,57468,2099-01-01
4,OFFICE DE TOURISME DE PLAINFAING,"TouristInformationCenter , LocalTouristOffice , PlaceOfInterest , PointOfInterest , TouristInformationCenter , urn:resource ]",Céline et Mélanie ont grand plaisir à vous accueillir et vous conseiller.,48.1480052,7.0094817,48.148#7.00948,PLAINFAING,44 Ter Habaurupt,88349,2099-01-01
5,ÉGLISE DE LA NATIVITÉ DE LA VIERGE MARIE,"CulturalSite , PlaceOfInterest , PointOfInterest , ReligiousSite , urn:resource ]","Église de style roman, construite à la fin du XIème siècle ou au début du siècle suivant.",49.135955281455,6.23665469206545,49.13595#6.23665,MEY,Place de lEglise,57467,2099-01-01
6,BIBLIOTHÈQUE LUTZELBOURG,"Library , CulturalSite , Library , PlaceOfInterest , PointOfInterest , urn:resource ]",Bibliothèque municipale du réseau de lecture du Pays de Phalsbourg.Catalogue en ligne !,48.7335143,7.2507663,48.73351#7.25076,LUTZELBOURG,8 rue Ackermann,57427,2099-01-01
7,JARDIN BOTANIQUE JEAN MARIE PELT,"Park , CulturalSite , ParkAndGarden , PlaceOfInterest , PointOfInterest , urn:resource ]","Avec plus de 12 000 espèces en culture, le Jardin Botanique Jean-Marie Pelt est un véritable musée des plantes qui témoigne de l’extraordinaire richesse du monde végétal

In [175]:
print(f"il reste : ", round(len(data_graph)/nb_ligne*100), "% du dataframe", "soit",len(data_graph), "lignes")

il reste :  94 % du dataframe soit 77147 lignes


ajouter code postale, département, région, num insee (pour une question de mise à jour et de coût, non utile pour l'utilisateur finale)

In [176]:
data_graph.isnull().sum()

lieu               0
type               0
description        0
latitude           0
longitude          0
latlong            0
ville              0
adresse            0
code_insee     61316
date_de_fin        0
dtype: int64

In [177]:
data_graph.to_csv('data_graph_def.csv', index=False)